In [5]:
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

device = torch.device("cuda")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

state_dict = torch.load("BART.pth", map_location=device)
model.load_state_dict(state_dict)

model.to(device)
model.eval()

C:\Users\MSI-Admin\AppData\Local\Temp\ipykernel_1684\4197821354.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("BART.pth", map_location=device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_n

In [6]:
def evaluate(model, tokenizer, input_text, device):
    model.eval()
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs)

    output = tokenizer.decode(outputs[0], skip_special_tokens=False)

    return tokenizer.decode(outputs[0], skip_special_tokens=True, truncation=False)

In [ ]:
import torch
import pandas as pd
import subprocess

def parse_movements(output_line):
    output_line = output_line.replace("End Task", "").strip()
    movements = []
    for part in output_line.split(","):
        part = part.strip()
        dx = dy = dz = 0
        if part.startswith("X +="):
            dx = int(part.split("+=")[1].strip())
        elif part.startswith("X -="):
            dx = -int(part.split("-=")[1].strip())
        elif part.startswith("Y +="):
            dy = int(part.split("+=")[1].strip())
        elif part.startswith("Y -="):
            dy = -int(part.split("-=")[1].strip())
        elif part.startswith("Z +="):
            dz = int(part.split("+=")[1].strip())
        elif part.startswith("Z -="):
            dz = -int(part.split("-=")[1].strip())
        else:
            continue
        movements.append((dx, dy, dz))
    return movements

try:
    df = pd.read_csv("movement_increments.csv")
except FileNotFoundError:
    df = pd.DataFrame(columns=["X", "Y", "Z"])

user = ""
with open("chat_output.txt", "a") as chat_file:
    while user != "-1":
        user = input("Enter command: ")
        if user != "-1":
            output = evaluate(model, tokenizer, user, device)
            print(f"User: {user}")
            print(f"Output: {output}")

            chat_file.write(f"User: {user}\n")
            chat_file.write(f"Output: {output}\n\n")

            new_rows = parse_movements(output)
            if new_rows:
                new_df = pd.DataFrame(new_rows, columns=["X", "Y", "Z"])
                df = pd.concat([df, new_df], ignore_index=True)
                df.to_csv("movement_increments.csv", index=False)
                print("CSV updated.")

                subprocess.run(["git", "add", "movement_increments.csv"])
                subprocess.run(["git", "commit", "-m", f"Update movement from: {user}"])
                subprocess.run(["git", "push"])
                print("Changes pushed to GitHub.")

print("Chat Ended.")

User: move left by 10 then right by 20 then up by 30
Output: X -= 10, X += 20, Z += 30, End Task
CSV updated.
User: move left by 10 up by 20 right by 30
Output: X -= 10, X += 30, End Task
CSV updated.
User: up by 20
Output: Z += 20, End Task
CSV updated.
User: down by 10
Output: Z -= 10, End Task
CSV updated.
User: left by 15
Output: X -= 15, End Task
CSV updated.
User: right by 30
Output: X += 30, End Task
CSV updated.
User: right by 13
Output: X += 13, End Task
CSV updated.
Chat Ended.
